# 3 重抽样方法

In [1]:
library(ISLR)

Warning message:
"package 'ISLR' was built under R version 3.3.3"

In [2]:
set.seed(1)

In [3]:
# 从原始的392个观测中随机抽取196个观察作为训练集
train <- sample(392, 196)

In [4]:
# 利用lm()函数的subset选项，在训练集上拟合线性回归模型
lm.fit=lm(mpg~horsepower,data=Auto,subset=train)

In [5]:
attach(Auto)

In [6]:
# 利用-train计算测试集上的MSE
mean((mpg-predict(lm.fit,Auto))[-train]^2)

[1] 26.14142

In [7]:
# 利用poly()函数计算二次和三次多项式回归在测试集上的MSE
lm.fit2=lm(mpg~poly(horsepower,2),data=Auto,subset=train)
mean((mpg-predict(lm.fit2,Auto))[-train]^2)

lm.fit3=lm(mpg~poly(horsepower,3),data=Auto,subset=train)
mean((mpg-predict(lm.fit3,Auto))[-train]^2)

[1] 19.82259

[1] 19.78252

## 3.1 留一交叉验证

如果用`glm()`函数拟合模型时没有设定`family`参数，那么它就跟`1m()`函数一样执行的是线性回归。接下来将用`glm()`函数来做线性回归，因为可以跟`cv.glm()`函数一起使用。使用`cv.glm()`函数需要安装`boot`包。

In [8]:
library(boot)

In [9]:
# cv.glm()函数会生成一个列表，delta向量中的两个数字为交叉验证误差（第二个值是经过调整的交叉验证误差）
glm.fit=glm(mpg~horsepower,data=Auto)
cv.err=cv.glm(Auto,glm.fit)
cv.err$delta

[1] 24.23151 24.23114

In [10]:
# 拟合一次到五次多项式回归模型，比较交叉验证误差；二次项测试MSE有大幅下降，更高次拟合效果没有显著提升
cv.error=rep(0,5)
for (i in 1:5){
 glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
 cv.error[i]=cv.glm(Auto,glm.fit)$delta[1]
 }
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

## 3.2 k折交叉验证法

使用留一交叉验证时，delta的两个数字完全相同，而使用k折CV方法时，delta的两个数字略有差别。第一个数字是标准k折CV估计值，第二个数字是偏差校正后的结果。

In [11]:
# 10折CV，计算一次到十次多项式回归模型的测试MSE
set.seed(17)
cv.error.10=rep(0,10)
for (i in 1:10){
 glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
 cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1]
 }
cv.error.10

[1] 24.20520 19.18924 19.30662 19.33799 18.87911 19.02103 18.89609 19.71201
 [9] 18.95140 19.50196

## 3.3 自助法

### 3.3.1 估计一个统计量的精度

In [12]:
# alpha.fn()函数根据输入的数据(X, Y)中的部分观测，计算alpha
alpha.fn=function(data,index){
 X=data$X[index]
 Y=data$Y[index]
 return((var(Y)-cov(X,Y))/(var(X)+var(Y)-2*cov(X,Y)))
 }

In [13]:
alpha.fn(Portfolio,1:100)

[1] 0.5758321

In [14]:
# 用sample()函数随机又放回地选取100个观测，创建一个新的自助法数据集，在新数据集上计算alpha。可以多次运行该行代码，在得到的alpha上计算标准差
set.seed(1)
alpha.fn(Portfolio,sample(100,100,replace=T))

[1] 0.5963833

In [15]:
# boot包中的boot()函数可以自动运用自助法计算标准差；对于原始数据，alpha估计值为0.5758，自助法标准差为0.0886
boot(Portfolio,alpha.fn,R=1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original        bias    std. error
t1* 0.5758321 -7.315422e-05  0.08861826

### 3.3.2 估计线性回归模型的精度

In [16]:
# boot.fn()函数，输入Auto数据集和观测序号的集合，返回线性回归模型的截距和斜率估计
boot.fn=function(data,index)
    return(coef(lm(mpg~horsepower,data=data,subset=index)))
boot.fn(Auto,1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [18]:
# 用boot()函数计算1000个截距和斜率项的自助法估计的标准误差；斜率标准差为0.86，截距标准差为0.0074
set.seed(1)
boot(Auto,boot.fn,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0269563085 0.859851825
t2* -0.1578447 -0.0002906457 0.007402954

In [19]:
# 用summary()函数得到用标准公式计算的标准误差
summary(lm(mpg~horsepower,data=Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81


标准公式计算的标准误差是基于某些假设的：1、公式依赖于未知的参数$\sigma^2$，即噪声方差。然后用RSS来估计$\sigma^2$。尽管标准误差的公式并不依赖于线性模型是正确的，但$\sigma^2$的估计却依赖于线性模型的正确性。由于数据存在非线性的关系，因此用线性拟合所得到的残差和$\sigma^2$都被放大了。2、标准公式假定$x_2$也是固定的(有点不现实)，所有的波动性都是由误差的$\epsilon_2$波动产生的。
而自助法并不依赖于这些假设，所以它相对于`summary()`函数，对标准误差的估计将更加准确。

In [20]:
# 二次多项式模型对数据的拟合效果更好，所以标准公式计算的标准误差与自助法更加接近
set.seed(1)

boot.fn=function(data,index)
     coefficients(lm(mpg~horsepower+I(horsepower^2),data=data,subset=index))

boot(Auto,boot.fn,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  6.098115e-03 2.0944855842
t2* -0.466189630 -1.777108e-04 0.0334123802
t3*  0.001230536  1.324315e-06 0.0001208339

In [21]:
summary(lm(mpg~horsepower+I(horsepower^2),data=Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),56.900099702,1.8004268063,31.60367,1.740911e-109
horsepower,-0.466189630,0.0311246171,-14.97816,2.289429e-40
I(horsepower^2),0.001230536,0.0001220759,10.08009,2.196340e-21
